In [1]:
import torch
from PIL import Image
from torchvision import transforms, models
from ModelAnalysis import ModelStatsAggregator, ModelStatAnalyser
from sys import version
from efficientnet_pytorch import EfficientNet
version

'3.9.5 (default, Aug 29 2021, 19:01:31) \n[GCC 9.3.0]'

In [2]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)


['adv_inception_v3',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'cspdarknet53',
 'cspresnet50',
 'cspresnext50',
 'deit_base_distilled_patch16_224',
 'deit_base_distilled_patch16_384',
 'deit_base_patch16_224',
 'deit_base_patch16_384',
 'deit_small_distilled_patch16_224',
 'deit_small_patch16_224',
 'deit_tiny_distilled_patch16_224',
 'deit_tiny_patch16_224',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'densenetblur121d',
 'dla34',
 'dla46_c',
 'dla46x_c',
 'dla60',
 'dla60_res2net',
 'dla60_res2next',
 'dla60x',
 'dla60x_c',
 'dla102',
 'dla102x',
 'dla102x2',
 'dla169',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'dpn68',
 'dpn

In [2]:
model = EfficientNet.from_pretrained('efficientnet-b0')

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /home/sultan/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 84.7MB/s]

Loaded pretrained weights for efficientnet-b0


In [2]:
model_dict = {
    'ssd': torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd'),
    'lenet': torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True),
    'yolov5s': torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True),
    'yolov5m': torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True),
    'yolov5l': torch.hub.load('ultralytics/yolov5', 'yolov5l', pretrained=True),
    'yolov5x': torch.hub.load('ultralytics/yolov5', 'yolov5x', pretrained=True),
    'alexnet': models.alexnet(pretrained=True, progress=True),
    'vgg_11': models.vgg11(pretrained=True, progress=True),
    'vgg_13': models.vgg13(pretrained=True, progress=True),
    'vgg_16': models.vgg16(pretrained=True, progress=True),
    'vgg_19': models.vgg19(pretrained=True, progress=True),
    'vgg_11_bn': models.vgg11_bn(pretrained=True, progress=True),
    'vgg_13_bn': models.vgg13_bn(pretrained=True, progress=True),
    'vgg_16_bn': models.vgg16_bn(pretrained=True, progress=True),
    'vgg_19_bn': models.vgg19_bn(pretrained=True, progress=True),
    'resnet_18': models.resnet18(pretrained=True, progress=True),
    'resnet_34': models.resnet34(pretrained=True, progress=True),
    'resnet_50': models.resnet50(pretrained=True, progress=True),
    'resnet_101': models.resnet101(pretrained=True, progress=True),
    'resnet_152': models.resnet152(pretrained=True, progress=True),
    'squeezenet_1_0': models.squeezenet1_1(pretrained=True, progress=True),
    'squeezenet_1_1': models.squeezenet1_0(pretrained=True, progress=True),
    'densenet_121': models.densenet121(pretrained=True, progress=True),
    'densenet_169': models.densenet169(pretrained=True, progress=True),
    'densenet_201': models.densenet201(pretrained=True, progress=True),
    'densenet_161': models.densenet161(pretrained=True, progress=True),
    'inception_v3': models.inception_v3(pretrained=True, progress=True),
    'googlenet': models.googlenet(pretrained=True, progress=True),
    'shufflenet_v2_x0_5': models.shufflenet_v2_x0_5(pretrained=True, progress=True),
    'shufflenet_v2_x1_0': models.shufflenet_v2_x1_0(pretrained=True, progress=True),
    'mobilenet_v2': models.mobilenet_v2(pretrained=True, progress=True),
    'mobilenet_v3_large': models.mobilenet_v3_large(pretrained=True, progress=True),
    'mobilenet_v3_small': models.mobilenet_v3_small(pretrained=True, progress=True),
    'resnext_50_32x4d': models.resnext50_32x4d(pretrained=True, progress=True),
    'resnext_101_32x8d': models.resnext101_32x8d(pretrained=True, progress=True),
    'wide_resnet_50_2': models.wide_resnet50_2(pretrained=True, progress=True),
    'wide_resnet_101_2': models.wide_resnet101_2(pretrained=True, progress=True),
    'mnasnet0_5': models.mnasnet0_5(pretrained=True, progress=True),
    'mnasnet1_0': models.mnasnet1_0(pretrained=True, progress=True),
}

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [3]:
# prepare sample inputs
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

input_image = Image.open(filename)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

uris = [
    'http://images.cocodataset.org/val2017/000000397133.jpg'
]
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

inputs = [utils.prepare_input(uri) for uri in uris]
ssd_input_batch = utils.prepare_tensor(inputs)

Using cache found in /home/sultan/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [4]:
stats_dict, _ = ModelStatAnalyser.get_models_stats_dict(model_dict, input_batch, ssd_input_batch)

Analysing ssd
Analysing lenet
Analysing yolov5s
Analysing yolov5m
Analysing yolov5l
Analysing yolov5x
Analysing alexnet
Analysing vgg_11
Analysing vgg_13
Analysing vgg_16
Analysing vgg_19
Analysing vgg_11_bn
Analysing vgg_13_bn
Analysing vgg_16_bn
Analysing vgg_19_bn
Analysing resnet_18
Analysing resnet_34
Analysing resnet_50
Analysing resnet_101
Analysing resnet_152
Analysing squeezenet_1_0
Analysing squeezenet_1_1
Analysing densenet_121
Analysing densenet_169
Analysing densenet_201
Analysing densenet_161
Analysing inception_v3
Analysing googlenet
Analysing shufflenet_v2_x0_5
Analysing shufflenet_v2_x1_0
Analysing mobilenet_v2
Analysing mobilenet_v3_large
Analysing mobilenet_v3_small
Analysing resnext_50_32x4d
Analysing resnext_101_32x8d
Analysing wide_resnet_50_2
Analysing wide_resnet_101_2
Analysing mnasnet0_5
Analysing mnasnet1_0


In [5]:
ModelStatsAggregator.get_aggregate_kernel_stats_as_percentages(stats_dict)

{'7': 0.011862396204033215,
 '1': 0.5776986951364176,
 '3': 0.39541320680110714,
 '5': 0.015025701858442072}

In [6]:
ModelStatsAggregator.get_aggregate_stride_stats_per_kernel(stats_dict)

{'7': Counter({'2': 17, '1': 13}),
 '1': Counter({'1': 1431, '2': 28}),
 '3': Counter({'1': 910, '2': 90}),
 '11': Counter({'4': 1}),
 '5': Counter({'1': 28, '2': 10})}

In [10]:
ModelStatsAggregator.get_aggregate_in_channel_stats_per_kernel(stats_dict)

{'7': {'3': 17, '160': 6, '192': 4, '128': 3},
 '1': {'1024': 185,
  '256': 180,
  '512': 157,
  '128': 69,
  '64': 58,
  '192': 47,
  '96': 40,
  '160': 38,
  '384': 36,
  '768': 36,
  '320': 33,
  '48': 31,
  '24': 26,
  '640': 24,
  '2048': 22,
  '480': 21,
  '832': 21,
  '1280': 19,
  '576': 19,
  '288': 17,
  '116': 17,
  '16': 15,
  '960': 14,
  '32': 13,
  '240': 13,
  '40': 13,
  '80': 11,
  '672': 11,
  '528': 10,
  '1152': 10,
  '72': 10,
  '232': 9,
  '120': 9,
  '144': 8,
  '58': 7,
  '1536': 6,
  '224': 6,
  '352': 6,
  '416': 6,
  '448': 6,
  '864': 6,
  '896': 6,
  '928': 6,
  '992': 6,
  '1056': 6,
  '1248': 6,
  '704': 5,
  '736': 5,
  '800': 5,
  '1344': 5,
  '1440': 5,
  '1632': 5,
  '544': 4,
  '608': 4,
  '1088': 4,
  '1120': 4,
  '1184': 4,
  '1216': 4,
  '1728': 4,
  '1312': 3,
  '1376': 3,
  '1408': 3,
  '1472': 3,
  '1504': 3,
  '1568': 3,
  '1600': 3,
  '1824': 3,
  '1664': 2,
  '1696': 2,
  '1760': 2,
  '1792': 2,
  '336': 2,
  '432': 2,
  '624': 2,
  '720': 

In [8]:
ModelStatsAggregator.get_aggregate_filter_stats_per_kernel(stats_dict)

{'7': {'64': 15, '192': 7, '160': 4, '96': 2, '128': 2},
 '1': {'128': 328,
  '1024': 174,
  '256': 168,
  '192': 134,
  '512': 113,
  '64': 64,
  '160': 42,
  '96': 41,
  '48': 38,
  '320': 33,
  '2048': 31,
  '24': 28,
  '384': 26,
  '32': 24,
  '640': 18,
  '116': 17,
  '16': 16,
  '80': 14,
  '576': 14,
  '40': 13,
  '255': 12,
  '240': 11,
  '72': 10,
  '144': 9,
  '58': 9,
  '232': 9,
  '120': 9,
  '960': 8,
  '112': 6,
  '1280': 6,
  '480': 5,
  '672': 4,
  '288': 4,
  '1152': 4,
  '768': 3,
  '1000': 2,
  '448': 2,
  '184': 2,
  '168': 2,
  '8': 2,
  '896': 1,
  '1056': 1,
  '200': 1,
  '88': 1},
 '3': {'32': 248,
  '256': 156,
  '512': 113,
  '48': 95,
  '128': 87,
  '64': 58,
  '1024': 33,
  '320': 27,
  '192': 25,
  '96': 25,
  '384': 20,
  '160': 17,
  '24': 12,
  '640': 10,
  '16': 9,
  '116': 9,
  '80': 5,
  '232': 5,
  '576': 5,
  '58': 4,
  '72': 4,
  '324': 3,
  '486': 3,
  '288': 3,
  '960': 3,
  '2048': 3,
  '208': 2,
  '224': 2,
  '144': 2,
  '240': 2,
  '184': 2,
 

In [9]:
ModelStatsAggregator.get_aggregate_filter_stats(stats_dict)

{'128': 417,
 '256': 324,
 '32': 272,
 '512': 226,
 '1024': 207,
 '192': 167,
 '64': 141,
 '48': 134,
 '96': 69,
 '160': 63,
 '320': 60,
 '384': 46,
 '24': 40,
 '2048': 34,
 '640': 28,
 '116': 26,
 '16': 25,
 '576': 25,
 '80': 19,
 '240': 18,
 '72': 18,
 '232': 14,
 '120': 14,
 '40': 13,
 '144': 13,
 '58': 13,
 '960': 13,
 '255': 12,
 '480': 9,
 '288': 9,
 '1152': 8,
 '1280': 7,
 '112': 6,
 '672': 6,
 '768': 4,
 '184': 4,
 '324': 3,
 '486': 3,
 '1000': 2,
 '448': 2,
 '168': 2,
 '8': 2,
 '200': 2,
 '88': 2,
 '208': 2,
 '224': 2,
 '896': 1,
 '1056': 1}